This notebook shows how a GCN/GraphSAGE model is trained to compute Node betweenness centrality on different graphs

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
import torch.nn as nn

class Net1(torch.nn.Module):
    def __init__(self, d1=16):
        super(Net1, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, d1)
        self.conv2 = GCNConv(d1, dataset.num_classes)
        self.d1 = d1
        

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # output as multiclass target
        #return F.log_softmax(x, dim=1)
        
        # output as regression target
        return x
    
    def __str__(self):
        return "Net1-gcn(%d,%d)-gcn(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                               dataset.num_classes)
    
    
class Net2(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, d1 )
        #self.conv2 = GCNConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    
    def __str__(self):
        return "Net2-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)

    
    
class Net3(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net3, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, d1 )
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    def __str__(self):
        return "Net3-SAGEgcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)


In [2]:
import networkx as nx
import time
import torch
from torch_geometric.data import DataLoader
import importlib
import torch
from torch_geometric.data import DataLoader
import networkx as nx
from torch_geometric.data import Data
import pickle

def loadDataset(collection, name=None):
    try:
        # import datasets
        themodule = importlib.import_module("torch_geometric.datasets")
        # get the function corresponding to collection
        method_to_call = getattr(themodule, collection)
        if name:
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except:
        # custom module
        method_to_call = globals()[collection]
       
        if name:
            
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
        


def transformMask(mask):
    train_mask = []
    i = 0
    for pick in mask:
        if pick[0]==1:
            train_mask.append(i)
        i+=1
    return train_mask


def shuffleTrainTestMasks(data, trainpct = 0.7):
    ysize = list(data.y.size())[0]
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[int(ysize*trainpct):] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
  

def shuffleTrainTestValMasks(data, trainpct = 0.7, valpct = 0.2):

    ysize = list(data.y.size())[0]
    #print("total ", ysize)
    #print(" train ",int(ysize*trainpct)-int(ysize*trainpct*valpct))
    #print(" val ",int(ysize*trainpct*valpct))
    #print(" test ",int(ysize*(1- trainpct) ))
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[:int(ysize*trainpct)] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    #print(" train sum ",data.train_mask.sum())
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    #print(" test sum ",data.test_mask.sum())
    
    # transform to list of indexes
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
    
    data.val_mask = data.train_mask[:int(ysize*trainpct*valpct)]
    data.train_mask = data.train_mask[int(ysize*trainpct*valpct):]

    
    #print(data.train_mask)
    #print(data.val_mask)
    #print(data.test_mask)
    
    

def trainTestEval(dataset, epochs=1, batch_size=32):
    global Net
    loader = DataLoader(dataset,  shuffle=False)
    i = 0
    print(loader)
    print(dir(loader))
    
    G = dataset.data
    print(G)
    start = time.time()


    # 1.  prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print("using ",device)
    model = Net.to(device)  
    data = G.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    # 2.  create a train_mask, and a test_mask (val_mask for further experiments)
    #shuffleTrainTestMasks(data)
    #shuffleTrainTestValMasks(data)
    shuffleTrainTestMasks(data)

    # 3. train some epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 25 == 0 :
            print("epoch-loss: ",epoch, loss)

    # 4. Model evaluation
    model.eval()
    #  classification in a multiclass setting
    #_, pred = model(data).max(dim=1)
    #correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    #acc = correct / data.test_mask.sum().item()
    #print('Accuracy: {:.4f}'.format(acc))


    # regression 
    pred = model(data)
    #print("target: ",data.y[data.test_mask])
    #print("prediction: ",pred[data.test_mask])
    #print(pred[data.test_mask].type())
    #print(data.y[data.test_mask].type())
    
    # prepare the normalized mean root squared error
    t = data.y[data.test_mask]
    y = pred[data.test_mask]
    nrmse = torch.sum((t - y) ** 2)/len(data.test_mask)
    nrmse = nrmse.sqrt()
    print("RMSE: ",nrmse)

    #m = torch.mean(t)
    #print("mean",m)
    #tmax = torch.max(t)
    #tmin = torch.min(t)
    #sd = tmax-tmin
    #print("sd",sd)
    #nrmse = (nrmse - m)/sd
    #print("NRMSE:",nrmse)


    endtime = time.time()
    print("Total train-test time: "+str(endtime-start))
    
    with open("results.txt","a") as f:
        print(dir(dataset))
        f.write("\n")
        f.write(str(model)+" " 
                +str(dataset.filename)+" "  
                +"nrmse: "+str(nrmse.item())+" " 
                +"total time: "+str(endtime-start) 
                +" negative vals?: "+str(False) 
                +"\n"
               )
    
    del model

    #i+=1
    #if i==1:
    #    break

In [3]:
class MyOwnDataset2():
    def __init__(self,  root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #print(self.data.num_features)
        self.num_features = self.data.num_features
        self.num_classes = 1
        f.close()
        
        

## Testing the Net for node Betweenness

In [4]:

dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/TUDataset_ENZYMES_11_nd.pickle')
global Net
Net=Net1(d1=20)
trainTestEval(dataset,  epochs=1)
del Net

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backend', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_slow_forward', '_state_dict_hooks', '_tracing_name', '_version', 'add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'half', 'load_state_dict', 'message', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'propaga

In [8]:

dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_1000_0_45_nb.pickle')
global Net
Net=Net1(d1=20)
trainTestEval(dataset,  epochs=1)
del Net

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backend', '_backward_hooks', '_buffers', '_forward_hooks', '_forward_pre_hooks', '_get_name', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_modules', '_named_members', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_slow_forward', '_state_dict_hooks', '_tracing_name', '_version', 'add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'half', 'load_state_dict', 'message', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'propaga

{'x': tensor([[-0.0587],
        [-0.0546],
        [-0.0559],
        [-0.0537],
        [-0.0569],
        [-0.0569],
        [-0.0544],
        [-0.0555],
        [-0.0551],
        [-0.0566],
        [-0.0566],
        [-0.0562],
        [-0.0572],
        [-0.0548],
        [-0.0547],
        [-0.0578],
        [-0.0541],
        [-0.0573],
        [-0.0551],
        [-0.0589],
        [-0.0539],
        [-0.0534],
        [-0.0561],
        [-0.0544],
        [-0.0530],
        [-0.0557],
        [-0.0528],
        [-0.0556],
        [-0.0556],
        [-0.0566],
        [-0.0569],
        [-0.0546],
        [-0.0543],
        [-0.0558],
        [-0.0563],
        [-0.0548],
        [-0.0550],
        [-0.0571],
        [-0.0545],
        [-0.0554],
        [-0.0543],
        [-0.0552],
        [-0.0541],
        [-0.0568],
        [-0.0536],
        [-0.0571],
        [-0.0549],
        [-0.0569],
        [-0.0535],
        [-0.0568],
        [-0.0562],
        [-0.0540],
      

In [5]:
del Net

In [39]:

dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_1000_0_45_nb.pickle')
global Net
Net=Net2(d1=30,d2=10)
trainTestEval(dataset,  epochs=1)
del Net

['_DataLoader__initialized', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_sampler', 'batch_size', 'collate_fn', 'dataset', 'drop_last', 'num_workers', 'pin_memory', 'sampler', 'timeout', 'worker_init_fn']
Data(edge_index=[2, 449110], x=[1000, 1], y=[1000])
epoch-loss:  0 tensor(0.0061, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(5.6608e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(3.4854e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(1.4919e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(9.4465e-07, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor

In [40]:
dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_1000_0_15_nb.pickle')
global Net
Net=Net2(d1=30,d2=50)
trainTestEval(dataset, epochs=200)
del Net

['_DataLoader__initialized', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_sampler', 'batch_size', 'collate_fn', 'dataset', 'drop_last', 'num_workers', 'pin_memory', 'sampler', 'timeout', 'worker_init_fn']
Data(edge_index=[2, 149596], x=[1000, 1], y=[1000])
epoch-loss:  0 tensor(0.0105, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(3.9370e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(7.9181e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(5.0373e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(2.3855e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor

In [23]:
dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_1000_0_15_nb.pickle')
global Net
Net=Net2(d1=30,d2=50)
trainTestEval(dataset,  epochs=200)
del Net

['_DataLoader__initialized', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_sampler', 'batch_size', 'collate_fn', 'dataset', 'drop_last', 'num_workers', 'pin_memory', 'sampler', 'timeout', 'worker_init_fn']
Data(edge_index=[2, 149596], x=[1000, 1], y=[1000])
epoch-loss:  0 tensor(0.0089, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(3.8539e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(2.3776e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(1.3707e-06, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(6.1590e-07, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor

In [61]:
dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_100_0_15_nb.pickle')
global Net
Net=Net3(d1=30,d2=50)
trainTestEval(dataset,  epochs=200)
del Net

['_DataLoader__initialized', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'batch_sampler', 'batch_size', 'collate_fn', 'dataset', 'drop_last', 'num_workers', 'pin_memory', 'sampler', 'timeout', 'worker_init_fn']
Data(edge_index=[2, 1542], x=[100, 1], y=[100])
epoch-loss:  0 tensor(0.0187, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(5.2695e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(2.9153e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(2.1987e-05, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(1.8023e